In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Libraries and Files Imports

In [ ]:
from string import punctuation
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from zipfile import ZipFile 
import keras
from keras.models import Sequential
from keras.layers import LSTM
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, Dropout, Activation, Input
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras import initializers, regularizers, constraints, optimizers, layers
from sklearn.model_selection import train_test_split
%matplotlib inline

In [ ]:
filename_1 = "/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip"
filename_2 = "/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip"
filename_3= "/kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip"
filename_4="/kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip"
   
with ZipFile(filename_1, 'r') as zip: 
    zip.extractall()
    zip.read('test.csv')
    test_data=pd.read_csv('test.csv')
with ZipFile(filename_2, 'r') as zip: 
    zip.extractall()
    zip.read('train.csv')
    train_data=pd.read_csv('train.csv')
with ZipFile(filename_3, 'r') as zip: 
    zip.extractall()
    zip.read('sample_submission.csv')
    sample_submission=pd.read_csv('sample_submission.csv')
with ZipFile(filename_4, 'r') as zip: 
    zip.extractall()
    zip.read('test_labels.csv')
    test_labels=pd.read_csv('test_labels.csv')

In [ ]:
test_data.head()

In [ ]:
train_data.head()

In [ ]:
test_labels.head()

In [ ]:
sample_submission.head()

# Data Preprocessing

In [ ]:
train_data.info()

In [ ]:
train_data.head()

In [ ]:
labels=['toxic','severe_toxic','obscene','threat','insult','identity_hate']
for i in labels:
    print('The no of {} comments in file is {}'.format(i,train_data[i].sum()))

In [ ]:
def data_cleaning(text):
    text = text.lower()
    text= ''.join([c for c in text if c not in punctuation])
    text = word_tokenize(text)
    stop_words = stopwords.words('english')
    text=[i for i in text if i not in stop_words]
    text = [i for i in text if i.isalpha()]
    
    lem=WordNetLemmatizer()
    normalized_text=[]
    for word in text:
        normalized_word=lem.lemmatize(word,'v')
        normalized_text.append(normalized_word)

    return normalized_text
    

In [ ]:
data_1=train_data[train_data['toxic']==1]
data_2=train_data[train_data['toxic']==0][:20000]
sample_data=pd.concat([data_1,data_2],axis=0)
sample_data.sort_index(inplace=True)

In [ ]:
sample_data['comment_text']=sample_data['comment_text'].apply(data_cleaning)

In [ ]:
length=[]
for i in sample_data['comment_text']:
   length.append(len(i)) 
sample_data['length']=length
sample_data=sample_data[sample_data['length']>4]
sample_data=sample_data[sample_data['length']<500]
sample_data.head()

In [ ]:
max_length=200
max_features=20000
def data_preprocessing(data,max_features=20000,max_len=200):
    tokenizer=Tokenizer(num_words=max_features)
    tokenizer.fit_on_texts(sample_data['comment_text'])
    data=tokenizer.texts_to_sequences(data)
    data = pad_sequences(data,max_len)
    return data

In [ ]:
labels=['toxic','severe_toxic', 'obscene' ,'threat', 'insult','identity_hate']
final_data=pd.DataFrame(data_preprocessing(sample_data['comment_text']))
final_data['index']=sample_data.index
final_data.set_index("index",inplace=True)
final_data[labels]=sample_data[labels]
final_data.head()

In [ ]:
labels=['toxic','severe_toxic','obscene','threat','insult','identity_hate']
for i in labels:
    print('The no of {} comments in file is {}'.format(i,final_data[i].sum()))

In [ ]:
toxic_a=final_data[final_data['toxic']==1]
toxic_b=final_data[final_data['toxic']==0][:20000]
toxic_data=pd.concat((toxic_a,toxic_b),axis=0).sort_index()


severe_a=final_data[final_data['severe_toxic']==1]
severe_b=final_data[final_data['severe_toxic']==0][:2000]
severe_data=pd.concat((severe_a,severe_b),axis=0).sort_index()


obscene_a=final_data[final_data['obscene']==1]
obscene_b=final_data[final_data['obscene']==0][:8000]
obscene_data=pd.concat((severe_a,severe_b),axis=0).sort_index()


threat_a=final_data[final_data['threat']==1]
threat_b=final_data[final_data['threat']==0][:600]
threat_data=pd.concat((threat_a,threat_b),axis=0).sort_index()


insult_a=final_data[final_data['insult']==1]
insult_b=final_data[final_data['insult']==0][:8000]
insult_data=pd.concat((threat_a,threat_b),axis=0).sort_index()


hate_a=final_data[final_data['identity_hate']==1]
hate_b=final_data[final_data['identity_hate']==0][:1500]
hate_data=pd.concat((hate_a,hate_b),axis=0).sort_index()


In [ ]:
test_data['comment_text']=test_data['comment_text'].apply(data_cleaning)

In [ ]:
test_df=pd.DataFrame(data_preprocessing(test_data['comment_text']))
test_df.head()

In [ ]:
test_df.to_csv('processed_test_data.csv',index=False)

In [ ]:
pd.read_csv('processed_test_data.csv')

# Traning And Prediction

In [ ]:
class lstm_model(object):
    def __init__(self,max_features=20000):
        self.max_features=max_features
        self.model=Sequential()
        self.model.add(Input(shape=max_length,))
        self.model.add(Embedding(self.max_features,128))
        self.model.add(Bidirectional(LSTM(50, return_sequences=True,recurrent_dropout=0.2)))
        self.model.add(Bidirectional(LSTM(50, return_sequences=True,recurrent_dropout=0.2)))
        self.model.add(GlobalMaxPool1D())
        self.model.add(Dense(1,activation='sigmoid'))
        self.model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
        
    def fit(self,X,y,batchsize,epochs):

        self.model.fit(X,y, batch_size=batchsize, epochs=epochs, validation_split=0.2)
        
    def predict(self,data):
        return self.model.predict(data)
        
    
    def predict_classes(self,data):
        return self.model.predict_classes(data)
        
    

In [ ]:
toxic_model=lstm_model()
toxic_model.fit(X=toxic_data.drop(labels,axis=1), y=toxic_data['toxic'],batchsize=100,epochs=4)

In [ ]:
toxic_predictions=toxic_model.predict_classes(test_df)

In [ ]:
severe_model=lstm_model()
severe_model.fit(X=severe_data.drop(labels,axis=1), y=severe_data['severe_toxic'],batchsize=100,epochs=4)


In [ ]:
severe_predictions=severe_model.predict_classes(test_df)


In [ ]:
obscene_model=lstm_model()
obscene_model.fit(X=obscene_data.drop(labels,axis=1), y=obscene_data['obscene'],batchsize=100,epochs=4)


In [ ]:
obscene_predictions=obscene_model.predict_classes(test_df)

In [ ]:
threat_model=lstm_model()
threat_model.fit(X=threat_data.drop(labels,axis=1), y=threat_data['threat'],batchsize=100,epochs=20)


In [ ]:
threat_predictions=threat_model.predict_classes(test_df)

In [ ]:
insult_model=lstm_model()
insult_model.fit(X=insult_data.drop(labels,axis=1), y=insult_data['insult'],batchsize=100,epochs=10)


In [ ]:
insult_predictions=insult_model.predict_classes(test_df)

In [ ]:
hate_model=lstm_model()
hate_model.fit(X=hate_data.drop(labels,axis=1), y=hate_data['identity_hate'],batchsize=100,epochs=4)


In [ ]:
hate_predictions=hate_model.predict_classes(test_df)

# Submission

In [ ]:
sample_submission['toxic']=toxic_predictions
sample_submission['severe_toxic']=severe_predictions
sample_submission['obscene']=obscene_predictions
sample_submission['threat']=threat_predictions
sample_submission['insult']=insult_predictions
sample_submission['identity_hate']=hate_predictions
sample_submission.head()

In [ ]:
sample_submission.to_csv('submission.csv',index=False)